In [8]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import bitstring
from qiskit_aer import AerSimulator

from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

from qiskit.circuit.library import UnitaryGate

In [9]:
def code():                                     #intialize |00> state
    qr = QuantumRegister(15,"q")
    cbits = ClassicalRegister(10,"c")
    qc = QuantumCircuit(qr, cbits)

    qc.x(0)
    qc.x(1)
    qc.x(2)
    qc.x(7)
    qc.x(11)
    qc.x(12)

    qc.cx(12,3)
    qc.cx(7,13)
    qc.cx(11,14)

    qc.cx(0,3)
    qc.cx(1,3)
    qc.cx(2,3)

    qc.cx(7,4)
    qc.cx(7,5)
    qc.cx(7,6)

    qc.cx(11,8)
    qc.cx(11,9)
    qc.cx(11,10)

    qc.cx(12,3)
    qc.cx(7,13)
    qc.cx(11,14)

    qc.h(12)
    qc.measure(12,0)
    qc.measure(13,0)
    qc.measure(14,0)

    qc.reset(12)
    qc.reset(13)
    qc.reset(14)
    ####################################################
    for i in range(12):
        qc.cx(i,i+4)
    ####################################################
    qc.h(12)

    qc.cx(12,4)
    qc.cx(7,13)
    qc.cx(12,7)
    qc.cx(4,13)
    qc.cx(12,5)
    qc.cx(6,13)
    qc.cx(12,6)
    qc.cx(5.13)

    qc.h(12)
    qc.measure(12,0)
    qc.measure(13,0)

    qc.reset(12)
    qc.reset(13)
    ####################################################
    qc.h(12)

    qc.cx(12,13)
    qc.cx(8,13)
    qc.cx(10,12)
    qc.cx(9,12)
    qc.cx(11,13)
    qc.cx(12,13)

    qc.h(12)

    qc.measure(12,0)
    qc.measure(13,0)

    qc.reset(12)
    qc.reset(13)
    
    return qc

def X_L(qc: QuantumCircuit, pos: int):
    qc.x(3)
    qc.x(10)
    if pos == 0:
        qc.x(0)
        qc.x(11)
    elif pos == 1:
        qc.x(1)
        qc.x(9)

def Z_L(qc: QuantumCircuit, pos: int):
    qc.z(0)
    qc.x(9)
    if pos == 0:
        qc.x(3)
        qc.x(11)
    elif pos == 1:
        qc.x(1)
        qc.x(10)


